In [ ]:
import os, sys
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../utils')
import pandas as pd
import numpy as np
from queryHelper import prodFetch, adbFetch
from datetime import timedelta, datetime
import psycopg2
import sqlalchemy
import time

In [ ]:
conn_params = {
    'dbname': 'operations_manager_prod',
    'user': 'XXXX',
    'password': 'XXXXX',
    'host': 'operation.replica.upgrid.in',
    'port': '5432'
}

In [ ]:
# Function to execute a query with retry mechanism
def execute_query_with_retries(query, conn_params, retries=10, delay=2):
    attempt = 0
    while attempt < retries:
        try:
            # Connect to the database
            with psycopg2.connect(**conn_params) as conn:
                df = pd.read_sql(query, conn)
            print("Query executed successfully.")
            return df
        except Exception as e:
            attempt += 1
            print(f"Attempt {attempt} failed: {e}")
            time.sleep(delay)
            delay *= 2  # Exponential backoff
    raise SystemExit("Query failed after multiple retries. Stopping execution.")


In [ ]:
query1 = '''with v1 as 
(
select
    TO_CHAR((t.created_at + INTERVAL '330 minute')::date, 'YYYY-MM') as month,
    EXTRACT(WEEK FROM (t.created_at + INTERVAL '330 minute')::date) AS week_number,
    (case 
        when (EXTRACT(DAY FROM (t.created_at + INTERVAL '330 minute')::date) >= 1 and EXTRACT(DAY FROM (t.created_at + INTERVAL '330 minute')::date) <= 10) then '1-10' 
        when (EXTRACT(DAY FROM (t.created_at + INTERVAL '330 minute')::date) >= 11 and EXTRACT(DAY FROM (t.created_at + INTERVAL '330 minute')::date) <= 20) then '11-20'
        when EXTRACT(DAY FROM (t.created_at + INTERVAL '330 minute')::date) >= 21 then '21+' 
    end) day_cohort,
    to_char((t.created_at + INTERVAL '330 minute')::date, 'Day') as day,
    (t.created_at + INTERVAL '330 minute')::date AS date,
    t.created_at + INTERVAL '330 minute' AS created_at,
    t.updated_at + INTERVAL '330 minute' AS updated_at,
    TO_CHAR(t.created_at + INTERVAL '330 minutes', 'HH24') as hour,
    t.slug ticket_id,
    tasks.status ticket_status,
    (case when t.complainant_type = 1 then 'driver' else 'partner' end) complainant_type,
    t.complainant_id complainant_id,
    ic.id category_id,
    ic.name category_name,
    t.issue_id issue_id,
    i1.name issue_name,
    concat(ic.name,' - ',i1.name) category_issue,
    t.complainant_name complainant_name,
    t.zone complainant_zone,
    t.source,
    COALESCE(t.calling_number, '') calling_number,
    tasks.id taskId,
    COALESCE((case when iar.assignment_rule_type like '%TeamRole%' then tatr.team else NULL end),' ') assigned_to_team,
    COALESCE((case when iar.assignment_rule_type like '%TeamRole%' then tatr.role else NULL end ), '') assigned_to_role,
    COALESCE(tasks.location, '') taskLocation,
    COALESCE(x.rejectionCount,0) taskRejectionCount,
    COALESCE(x.rejectionReasons,'') rejectionReasons,
    COALESCE(x.rejectionRemarks,'') rejectionRemarks,
    battery_ids,
    split_part(battery_ids, ',', 1) AS batteryid_1,
    split_part(battery_ids, ',', 2) AS batteryid_2,
    charger_ids,
    partner_id, 
    driver_id
from tickets t
left join issues i1 on t.issue_id = i1.id
left join issue_categories ic on ic.id = i1.issue_category_id
left join tasks on tasks.id = t.task_id
left join issue_assignment_rules iar on iar.issue_id = t.issue_id
left join tickets_assignment_rules_team_roles tatr on tatr.id = iar.assignment_rule_id
left join (
with v1 as 
(
select loggable_type, loggable_id, to_value, count(id) rejectionCount, string_agg(reason, E' >> ') rejectionReasons, string_agg(remark, E' >> ') rejectionRemarks from logs
where to_value = 'rejected'
group by 1,2,3
)
select * from v1) x on x.loggable_id = t.task_id
where t.deleted_at is null and t.created_at >= '20240701' and t.created_at < '20240801'
)
select * from v1
'''

In [ ]:
# Function to fetch data in 5-day intervals
def fetch_data_in_batches(start_date, end_date, conn_params):
    # Convert string dates to datetime objects
    start_date = datetime.strptime(start_date, '%Y%m%d')
    end_date = datetime.strptime(end_date, '%Y%m%d')

    # Initialize a list to store dataframes
    dataframes = []

    # Iterate in 15-day intervals
    current_start = start_date
    while current_start < end_date:
        current_end = min(current_start + timedelta(days=3), end_date)

        # Format the date range for the query
        date_condition = (f"t.created_at >= '{current_start.strftime('%Y-%m-%d')}' "
                          f"and t.created_at < '{current_end.strftime('%Y-%m-%d')}'")

        # Adjust the query with the date condition
        query = query1.replace("t.created_at >= '20240701' and t.created_at < '20240801'", date_condition)

        print(f"Fetching data for range: {current_start.strftime('%Y-%m-%d')} to {current_end.strftime('%Y-%m-%d')}")

        # Execute the query and append the result to the list
        try:
            df = execute_query_with_retries(query, conn_params)
            dataframes.append(df)
        except Exception as e:
            print(f"Failed to fetch data for range {current_start} to {current_end}: {e}")

        # Move to the next interval
        current_start = current_end

    # Concatenate all dataframes into a single dataframe
    final_dataframe = pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()
    time.sleep(1)
    return final_dataframe

# Define the start and end date for the batch process
start_date = '20241001'
end_date = '20250213'

# Fetch the data in 15-day intervals
final_dataframe = fetch_data_in_batches(start_date, end_date, conn_params)

# Display the final dataframe
print(final_dataframe)

In [ ]:
dfTickets = final_dataframe.copy()
del final_dataframe

In [ ]:
dfTickets

In [ ]:
query2 = """with v1 as 
(
select
    t.slug ticket_id,
    1 as escalated_flag
from
    logs l
left join tickets t on t.task_id = l.loggable_id
left join tasks on tasks.id = t.task_id
left join issues i on i.id = t.issue_id
left join issue_categories ic on ic.id = i.issue_category_id
left join issue_assignment_rules iar on iar.issue_id = t.issue_id
left join tickets_assignment_rules_relationship_managers tarm on tarm.id = iar.assignment_rule_id
left join tickets_assignment_rules_team_roles tatr on tatr.id = iar.assignment_rule_id
where  to_value in ('escalated') and t.deleted_at is null
)
select * from v1"""


# Function to execute the query with retry mechanism
def execute_query_with_retries(query, retries=5, delay=2):
    attempt = 0
    while attempt < retries:
        try:
            # Connect to the database
            with psycopg2.connect(**conn_params) as conn:
                dfTickets = pd.read_sql(query, conn)
            print("Query executed successfully.")
            return dfTickets
        except Exception as e:
            attempt += 1
            print(f"Attempt {attempt} failed: {e}")
            time.sleep(delay)
            delay *= 2  # Exponential backoff
    raise SystemExit("Query failed after multiple retries. Stopping execution.")

# Execute the query with retry logic
try:
    dfEscalation = execute_query_with_retries(query2)
except Exception as e:
    print("Final attempt failed:", e)


dfEscalation

In [ ]:
dfTickets = dfTickets.merge(dfEscalation, on = 'ticket_id', how = 'left')

In [ ]:
dfTickets['escalated_flag'] = dfTickets['escalated_flag'].fillna(0) 

In [ ]:
dfTickets

In [ ]:
del dfEscalation

In [ ]:
# conn = psycopg2.connect(dbname = "operations_manager_prod", user = "ankit_das", password = "Ankit@12345", host = "operation.replica.upgrid.in", port = "5432")

# print('PostgreSQL Connection Established')

# query3 = '''with v1 as 
# (
# select l.loggable_type, l.loggable_id, l.attr, l.from_value, l.to_value, l.reason, l.remark, l.event_name, l.done_by,  l.created_at + INTERVAL '330 minute' created_at, l.updated_at + INTERVAL '330 minute' updated_at,
# COALESCE(t.parent_id,l.loggable_id) parent_Id
# from logs l
# left join tasks t on l.loggable_id = t.id 
# where deleted_at is null  and (t.created_at + INTERVAL '330 minute')::date >= '20240801' and l.to_value not in ('dropped')
# ),
# v2 as (
# select *, rank() over(partition by parent_id order by created_at desc) rank from v1
# )
# select * from v2 where rank = 1
# '''

# dfTat = pd.read_sql(query3, conn)

# conn.close()

In [ ]:
# dfTat['created_at'] = pd.to_datetime(dfTat['created_at'])
# dfTat['updated_at'] = pd.to_datetime(dfTat['updated_at'])
# dfTat

In [ ]:
# dfTat[dfTat['taskid']== 719823]

In [ ]:
# dfTat.rename({'parent_id':'taskid'}, axis = 1, inplace = True)

In [ ]:
# dfTat

In [ ]:
# dfTickets = dfTickets.merge(dfTat, on = 'taskid', how = 'left' )

In [ ]:
# dfUsers = prodFetch("""select employeeId temp, (case when teams in ('[]') then role else teams end) team, roleV2 from users 
# where employeeId not like 'D%' or employeeId not like 'P%'""")

In [ ]:
# replace_dict = {'[]':'', '[':'',']':'','"':''}

In [ ]:
# dfUsers['roleV2'] = dfUsers['roleV2'].fillna('')

In [ ]:
# for old_char, new_char in replace_dict.items():
#     dfUsers['team'] = dfUsers['team'].str.replace(old_char, new_char)

In [ ]:
# dfUsers['assinged_to_team'] = dfUsers['team'] + " - " + dfUsers['roleV2']

In [ ]:
# del dfUsers['team']
# del dfUsers['roleV2']

In [ ]:
# dfUsers

In [ ]:
# dfTat['temp'] = dfTat['done_by'].str[:8].str.replace(' ', '')

In [ ]:
# dfTat = dfTat.merge(dfUsers, on = 'temp', how = 'left')

In [ ]:
# dfTat

In [ ]:
# del dfTat['temp']
# dfTat

In [ ]:
# dfTickets = dfTickets.merge(dfTat, on = 'taskid', how = 'left')

In [ ]:
dfTickets.info()

In [ ]:
# del dfTickets['loggable_type']
# del dfTickets['loggable_id']
# del dfTickets['attr']
# del dfTickets['from_value']
# del dfTickets['to_value']
# del dfTickets['reason']
# del dfTickets['remark']
# del dfTickets['event_name']
# del dfTickets['rank']
# del dfTickets['created_at_y']

In [ ]:
dfTickets.rename({'updated_at_y':'log_timestamp','done_by':'last_assiged_to', 'created_at_x':'created_at','updated_at_x':'updated_at', 'assinged_to_team':'last_assigned_to_team'}, axis = 1, inplace = True)

In [ ]:
dfTickets.info()

In [ ]:
dfTickets['created_at'] = pd.to_datetime(dfTickets['created_at'])
dfTickets['updated_at'] = pd.to_datetime(dfTickets['updated_at'])
# dfTickets['log_timestamp'] = pd.to_datetime(dfTickets['log_timestamp'])

In [ ]:
dfTickets

In [ ]:
# dfTickets['tat_in_hrs'] = np.where(dfTickets['ticket_status']== "completed", ((dfTickets['log_timestamp'] - dfTickets['created_at']).dt.total_seconds()/3600).round(1), ((pd.Timestamp.now() - dfTickets['created_at']).dt.total_seconds()/3600).round(1))

In [ ]:
# dfTickets['tat_status'] = np.where(dfTickets['tat_in_hrs'] <= dfTickets['sla_in_hrs'], "Within_tat", "Outside_tat")

In [ ]:
# dfTickets

In [ ]:
dfTickets[dfTickets['ticket_id']=='D240805-116967']

In [ ]:
time.sleep(10)

In [ ]:
query3 = '''with v1 as 
(
select t.slug ticket_id, coalesce(string_agg(rs.name, ' >> '),'No Issue Found') resolution , string_agg(rt.remark, ' >>') remark from tickets t
left join issues i on t.issue_id = i.id
left join (select id, parent_id, type  from tasks where type in ('Tasks::SelectResolution')) ta on t.task_id = ta.parent_id
left join resolutions_tasks rt on rt.task_id = ta.id
left join resolutions rs on rs.id = rt.resolution_id
group by 1
)
select * from v1
'''
# Function to execute the query with retry mechanism
def execute_query_with_retries(query, retries=5, delay=2):
    attempt = 0
    while attempt < retries:
        try:
            # Connect to the database
            with psycopg2.connect(**conn_params) as conn:
                dfTickets = pd.read_sql(query3, conn)
            print("Query executed successfully.")
            return dfTickets
        except Exception as e:
            attempt += 1
            print(f"Attempt {attempt} failed: {e}")
            time.sleep(delay)
            delay *= 2  # Exponential backoff
    raise SystemExit("Query failed after multiple retries. Stopping execution.")

# Execute the query with retry logic
try:
    dfResolutions = execute_query_with_retries(query3)
except Exception as e:
    print("Final attempt failed:", e)

dfResolutions

In [ ]:
dfResolutions

In [ ]:
dfResolutions[dfResolutions['ticket_id']=='D240709-26943']

In [ ]:
dfDriverZone = prodFetch('''select id complainant_id, zoneId complainant_zone, livedate driver_ob_date from drivers''')

In [ ]:
dfTickets =  dfTickets.merge(dfDriverZone, on = 'complainant_id', how = 'left')

In [ ]:
dfTickets[dfTickets['ticket_id']== 'D240808-129143']

In [ ]:
dfTickets

In [ ]:
dfTickets.info()

In [ ]:
dfTickets['complainant_zone'] = np.where(dfTickets['complainant_type'] == 'driver', dfTickets['complainant_zone_y'], dfTickets['complainant_zone_x'])

In [ ]:
dfTickets

In [ ]:
del dfTickets['complainant_zone_x']
del dfTickets['complainant_zone_y']

In [ ]:
dfTickets['created_at'] = pd.to_datetime(dfTickets['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S'))
dfTickets['updated_at'] = pd.to_datetime(dfTickets['updated_at'].dt.strftime('%Y-%m-%d %H:%M:%S'))
dfTickets['batteryid_1'] = dfTickets['batteryid_1'].str.strip()
dfTickets['batteryid_2'] = dfTickets['batteryid_2'].str.strip()

In [ ]:
dfTickets['created_at'].info()

In [ ]:
dfResolutions

In [ ]:
dfTickets = dfTickets.merge(dfResolutions, on = 'ticket_id', how = 'left')

In [ ]:
dfTickets

In [ ]:
del dfResolutions

In [ ]:
# dfTickets['category_issue'].unique()

In [ ]:
# temp = ['Meter Not Working', 'Meter Stolen','Meter Not Working - Driver not ready to pay service change','Meter Not Working - Service Charge Applied']

In [ ]:
# dfVehicleIssue = dfTickets[dfTickets['issue_name'].isin(temp)]

In [ ]:
# dfVehicleIssue

In [ ]:
# dfVehicleIssue.to_clipboard()

In [ ]:
# write('https://docs.google.com/spreadsheets/d/19VdawFRRbRDneVaVvz4qS5_lxRq5BdaPZ54s9iVdxDA/edit?gid=1335881675#gid=1335881675', 'New Tickets', dfVehicleIssue)

In [ ]:
dfBatteries = prodFetch('''select id, manufacturerName, phase, batteryType from batteries''')

In [ ]:
dfBatteries.rename({'id':'batteryid_1'}, axis = 1, inplace = True)
dfTickets = dfTickets.merge(dfBatteries, on = 'batteryid_1', how = 'left')
dfBatteries.rename({'batteryid_1':'batteryid_2'}, axis = 1, inplace = True)
dfTickets = dfTickets.merge(dfBatteries, on = 'batteryid_2', how = 'left')

In [ ]:
dfBatteries

In [ ]:
del dfBatteries

In [ ]:
dfTickets

In [ ]:
colsToRename = {'manufacturerName_x':'b1_oem', 'phase_x': 'b1_phase', 'batteryType_x': 'b1_battery_type', 'manufacturerName_y':'b2_oem', 'phase_y' : 'b2_phase', 'batteryType_y':'b2_battery_type'}

dfTickets.rename(colsToRename, axis = 1, inplace = True)

In [ ]:
dfTickets

In [ ]:
# dfBatteryLogs = adbFetch('''select date(date_add(createdAt, interval 330 minute)) date, date_add(createdAt, interval 330 minute) createdAt, batteryId, serialNo, occupant, changedBy from batteryLogs
# where date(createdAt) >= 20240829 and deletedAt is null''')

In [ ]:
# dfBatteryLogsCopy = dfBatteryLogs.copy()

In [ ]:
# del dfBatteryLogs

In [ ]:
# dfBatteryLogsCopy.info()

In [ ]:
# dfBatteryLogsCopy.rename({'createdAt':'created_at','batteryId':'batteryid_1'}, axis = 1, inplace = True)

In [ ]:
# dfTickets = dfTickets.sort_values(by = ['created_at', 'batteryid_1'])
# dfBatteryLogsCopy = dfBatteryLogsCopy.sort_values(by = ['created_at', 'batteryid_1'])

In [ ]:
# dfBatteryLogsCopy.info()

In [ ]:
# dfTickets = pd.merge_asof(dfTickets, dfBatteryLogsCopy, left_on = 'created_at', right_on = 'created_at' , by = 'batteryid_1', direction = 'nearest')

In [ ]:
# dfTickets

In [ ]:
# del dfTickets['date_y']
# del dfTickets['serialNo']
# del dfTickets['changedBy']

# dfTickets.rename({'occupant':'b1_occupant'}, axis = 1, inplace = True)

In [ ]:
# dfTickets

In [ ]:
# dfBatteryLogsCopy.rename({'batteryid_1':'batteryid_2'}, axis = 1, inplace = True)
# dfBatteryLogs.rename({'createdAt':'created_at'}, axis = 1, inplace = True)

In [ ]:
# dfTickets[dfTickets['battery_ids'] != ""]

In [ ]:
# dfTickets = dfTickets.sort_values(by = ['created_at', 'batteryid_2'])
# dfBatteryLogsCopy = dfBatteryLogsCopy.sort_values(by = ['created_at', 'batteryid_2'])

In [ ]:
dfTickets

In [ ]:
# dfTickets = pd.merge_asof(dfTickets,dfBatteryLogsCopy, on = 'created_at', by = 'batteryid_2', direction = 'nearest')

In [ ]:
dfTickets

In [ ]:
# del dfBatteryLogsCopy

In [ ]:
# del dfTickets['date']
# del dfTickets['serialNo']
# del dfTickets['changedBy']

# dfTickets.rename({'occupant':'b2_occupant', 'date_x': 'date'}, axis = 1, inplace = True)

In [ ]:
dfTickets.info()

In [ ]:
dfTickets

In [ ]:
query6 = '''
select id partner_id, zoneId partnerZone, status partnerStatus from partners
'''

In [ ]:
df6 = prodFetch(query6)

In [ ]:
dfTickets = dfTickets.merge(df6, on = 'partner_id', how = 'left')

In [ ]:
dfTickets

In [ ]:
del df6

In [ ]:
# del dfTickets['date']
# del dfTickets['serialNo']
# del dfTickets['changedBy']

# dfTickets.rename({'date_x':'date','occupant':'b2_occupant'}, axis = 1, inplace = True)

In [ ]:
dfPenaltyWallet = adbFetch('''select date_add(date, interval 1 day) date, driverId complainant_id, round(penaltyWallet/100,0) penalty_wallet from dailyDriversHistories
where date >= 20240930 and deletedAt is null and driverId like "D%"''')

In [ ]:
dfTickets['date'] = pd.to_datetime(dfTickets['date']).dt.date
dfTickets['hour'] = dfTickets['hour'].apply(np.float64)

In [ ]:
dfTickets = dfTickets.merge(dfPenaltyWallet, on = ['date', 'complainant_id'], how = 'left')

In [ ]:
dfTickets

In [ ]:
del dfPenaltyWallet

In [ ]:
dfTickets['penalty_wallet'] = dfTickets['penalty_wallet'].fillna(0)

In [ ]:
dfTickets['penalty_wallet'] = dfTickets['penalty_wallet'].apply(np.float64)

In [ ]:
dfTickets

In [ ]:
dfTickets_copy = dfTickets.copy()

In [ ]:
del dfTickets

In [ ]:
dfDriverDetails = prodFetch('''
select d.id complainant_id, d.clientid client_id, c.name client, (case when d.vehicleType in ('e-2w') then 'e-2w' else 'e-3w' end) vehicle_type,
(case when clientId not in ('BS00') then true else false end) b2b_2w_flag, (case when clientId in ('BS00') and d.vehicleType not in ('E-2w') then true else false end) b2c_3w_flag, (case when clientId in ('BS00') and d.vehicleType in ('E-2w') then true else false end) b2c_2w_flag, d.isBaaSDriver baaS_flag, (case when datediff(date_add(current_date(),interval -1 day), d.liveDate) <= 30 then '<= 30 days' 
    	  when (datediff(date_add(current_date(),interval -1 day), d.liveDate) between 31 and 90) then '31-90 days'
    	  when datediff(date_add(current_date(),interval -1 day), d.liveDate) > 90 and datediff(date_add(current_date(),interval -1 day), d.liveDate) <=365  then '91-365 days'
          when datediff(date_add(current_date(),interval -1 day), d.liveDate) > 365 then '365+ days'
     else 'NA'
     end) age_cohort, dl.source driver_source, dl.sourceId source_id, u.mobile, u.alternatemobile alternate_mobile, d.operatorId operator_id, u2.mobile operator_mobile, d.status driver_status, d.liveDate driver_live_date,
     (CASE WHEN d.status = 'active'
   AND d.isBatteryAvailable = 0
                        AND d.isDefaulter = 0
                        AND d.nonopsdays = 0
                        AND d.livedate IS NOT NULL
                        AND d.deletedat IS NULL
                        AND d.isBaaSDriver = 0 THEN
                        'to be activated'
                WHEN d.status = 'active'
                        AND d.isBatteryAvailable = 1
                        AND d.isDefaulter = 0
                        AND d.nonopsdays = 0
                        AND d.livedate IS NOT NULL
                        AND d.deletedat IS NULL
                        AND d.isBaaSDriver = 0 THEN
                        'Active'
                WHEN d.status = 'active'
                        AND d.isBatteryAvailable = 1
                        AND d.isDefaulter = 0
                        AND d.nonopsdays = 0
                        AND d.livedate IS NOT NULL
                        AND d.deletedat IS NULL
                        AND d.isBaaSDriver = 1 THEN
                        'Active Rental'
                WHEN d.status = 'active'
                        AND d.isBatteryAvailable = 1
                        AND d.isDefaulter = 0
                        AND d.nonopsdays > 0
                        AND d.livedate IS NOT NULL
                        AND d.deletedat IS NULL
                        AND d.isBaaSDriver = 0 THEN
                        'Absent'
                WHEN d.status = 'inactive'
                        AND d.isBatteryAvailable = 0
                        AND d.isDefaulter = 0
                        AND d.nonopsdays >= 0
                        AND d.livedate IS NOT NULL
                        AND d.deletedat IS NULL
                        AND d.isBaaSDriver = 0 THEN
                        'On Leave'
                WHEN d.status = 'inactive'
                        AND d.isBatteryAvailable = 0
                        AND d.isDefaulter = 0
                        AND d.nonopsdays = 0
                        AND d.livedate IS NULL
                        AND d.deletedat IS NULL
                        AND d.isBaaSDriver = 0 THEN
                        'To be onboarded'
                WHEN d.status = 'inactive'
                        AND d.isBatteryAvailable = 1
                        AND d.isDefaulter = 1
                        AND d.nonopsdays >= 0
                        AND d.livedate IS NOT NULL
                        AND d.deletedat IS NULL
                        AND d.isBaaSDriver = 1 THEN
                        'Defaulter Rental'
                WHEN d.status = 'inactive'
                        AND d.isBatteryAvailable = 1
                        AND d.isDefaulter = 1
                        AND d.nonopsdays >= 0
                        AND d.livedate IS NOT NULL
                        AND d.deletedat IS NULL
                        AND d.isBaaSDriver = 0 THEN
                        'Defaulter'
                WHEN d.status = 'inactive'
                        AND d.isBatteryAvailable = 0
                        AND d.isDefaulter = 1
                        AND d.nonopsdays >= 0
                        AND d.livedate IS NOT NULL
                        AND d.deletedat IS NULL
                        AND d.isBaaSDriver = 0 THEN
                        'Blocked'
                WHEN d.status = 'left'
                        AND d.isBatteryAvailable = 0
                        AND d.isDefaulter = 0
                        AND d.nonopsdays = 0
                        AND d.livedate IS NOT NULL
                        AND d.deletedat IS NULL
                        AND d.isBaaSDriver = 0 THEN
                        'To be Deboarded'
                WHEN d.status = 'left'
                        AND d.isBatteryAvailable = 0
                        AND d.isDefaulter = 0
                        AND d.nonopsdays = 0
                        AND d.livedate IS NOT NULL
                        AND d.deletedat IS NOT NULL
                        AND d.isBaaSDriver = 0 THEN
                        'Deboarded'
                ELSE
                        '-'
                END) driver_front_end_status
from drivers d
left join clients c on c.id = d.clientId
left join driverLeads dl on dl.id = d.driverLeadId
left join users u on u.employeeId = d.id
left join driverOperators op on op.id = d.operatorId
left join users u2 on u2.employeeId = d.operatorId
where d.id like 'D%'
''')

In [ ]:
dfDriverDetails['mobile'] = dfDriverDetails['mobile'].apply(lambda x: '{:.0f}'.format(x))
dfDriverDetails['alternate_mobile'] = dfDriverDetails['alternate_mobile'].apply(lambda x: '{:.0f}'.format(x))

In [ ]:
dfDriverDetails

In [ ]:
dfTickets_copy = dfTickets_copy.merge(dfDriverDetails, on = 'complainant_id', how = 'left')

In [ ]:
dfTickets_copy

In [ ]:
dfTickets_copy = dfTickets_copy.fillna("")

In [ ]:
dfTickets_copy

In [ ]:
del dfDriverDetails
del dfDriverZone

In [ ]:
dfTickets_copy.info()

In [ ]:
dfTickets_copy['date'] = pd.to_datetime(dfTickets_copy['date'])

dfTickets_copy.set_index('date', inplace=True)

In [ ]:
dfTickets_copy = dfTickets_copy.sort_values(by=['complainant_id', 'category_issue', 'created_at'])

In [ ]:
grouped_df_7D = dfTickets_copy[['complainant_id','category_issue','created_at']].groupby(['complainant_id', 'category_issue']).rolling('7D').count().reset_index().groupby(['complainant_id','category_issue','date']).agg(tickets=('created_at','max')).reset_index()

grouped_df_7D = grouped_df_7D.reset_index(drop=True)

grouped_df_7D.rename({'tickets':'L7Dtickets'}, axis = 1, inplace=True)

In [ ]:
grouped_df_7D

In [ ]:
grouped_df_14D = dfTickets_copy[['complainant_id','category_issue','created_at']].groupby(['complainant_id', 'category_issue']).rolling('14D').count().reset_index().groupby(['complainant_id','category_issue','date']).agg(tickets=('created_at','max')).reset_index()

grouped_df_14D = grouped_df_14D.reset_index(drop=True)

grouped_df_14D.rename({'tickets':'L14Dtickets'},axis = 1,inplace=True)

In [ ]:
grouped_df_30D = dfTickets_copy[['complainant_id','category_issue','created_at']].groupby(['complainant_id','category_issue']).rolling('30D').count().reset_index().groupby(['complainant_id','category_issue','date']).agg(tickets=('created_at','max')).reset_index()

grouped_df_30D = grouped_df_30D.reset_index(drop=True)

grouped_df_30D.rename({'tickets':'L30Dtickets'},axis = 1,inplace=True)

In [ ]:
grouped_df_60D = dfTickets_copy[['complainant_id','category_issue','created_at']].groupby(['complainant_id', 'category_issue']).rolling('60D').count().reset_index().groupby(['complainant_id','category_issue','date']).agg(tickets=('created_at','max')).reset_index()

grouped_df_60D = grouped_df_60D.reset_index(drop=True)

grouped_df_60D.rename({'tickets':'L60Dtickets'},axis = 1,inplace=True)

In [ ]:
grouped_df_90D = dfTickets_copy[['complainant_id','category_issue','created_at']].groupby(['complainant_id', 'category_issue']).rolling('90D').count().reset_index().groupby(['complainant_id','category_issue','date']).agg(tickets=('created_at','max')).reset_index()

grouped_df_90D = grouped_df_90D.reset_index(drop=True)

grouped_df_90D.rename({'tickets':'L90Dtickets'},axis = 1,inplace=True)

In [ ]:
grouped_df_90D

In [ ]:
dfTickets_copy = dfTickets_copy.merge(grouped_df_7D, on = ['date','complainant_id','category_issue'], how='left')
dfTickets_copy = dfTickets_copy.merge(grouped_df_14D, on = ['date','complainant_id','category_issue'], how='left')
dfTickets_copy = dfTickets_copy.merge(grouped_df_30D, on = ['date','complainant_id','category_issue'], how='left')
dfTickets_copy = dfTickets_copy.merge(grouped_df_60D, on = ['date','complainant_id','category_issue'], how='left')
dfTickets_copy = dfTickets_copy.merge(grouped_df_90D, on = ['date','complainant_id','category_issue'], how='left')

In [ ]:
dfTickets_copy

In [ ]:
del grouped_df_7D
del grouped_df_14D
del grouped_df_30D
del grouped_df_60D
del grouped_df_90D

In [ ]:
# df_copy[(df_copy['ticketTypeSubType']=='Charger Issue - Charger is not working')&(df_copy['complainantId']=='P2261')]

For Ops Drivers & Transactions

In [ ]:
# # Zonal daily Ops Drivers

# query2 = '''
# select t.date, d.zoneId complainant_zone, 
#     count(distinct(case when d.clientId not in ('BS00') then driverId end)) zone_day_B2B_2W_ops, 
#     count(distinct(case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) zone_day_B2C_3W_ops, 
#     count(distinct(case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) zone_day_B2C_2W_ops,
#     count((case when d.clientId not in ('BS00') then driverId end)) zone_day_B2B_2W_txns, 
#     count((case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) zone_day_B2C_3W_txns, 
#     count((case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) zone_day_B2C_2W_txns
# from transactions t
# left join drivers d on d.id = t.driverId
# where t.date >= 20240701 and t.date <= current_date() and t.deletedAt is null
# group by 1,2
# '''

In [ ]:
# df2 = prodFetch(query2)

In [ ]:
# df2['date'] = pd.to_datetime(df2['date'])
# dfTickets_copy = dfTickets_copy.merge(df2, on = ['date','complainant_zone'], how = 'left')

In [ ]:
# # Zonal weekly Ops Drivers

# query3 = '''
# select week(t.date,1) week_number, d.zoneId complainant_zone, 
#     count(distinct(case when d.clientId not in ('BS00') then driverId end)) zone_week_B2B_2W_ops, 
#     count(distinct(case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) zone_week_B2C_3W_ops, 
#     count(distinct(case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) zone_week_B2C_2W_ops,
#     count((case when d.clientId not in ('BS00') then driverId end)) zone_week_B2B_2W_txns, 
#     count((case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) zone_week_B2C_3W_txns, 
#     count((case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) zone_week_B2C_2W_txns 
# from transactions t
# left join drivers d on d.id = t.driverId
# where t.date >= 20240701 and t.date <= current_date() and t.deletedAt is null
# group by 1,2
# '''

In [ ]:
# df3 = prodFetch(query3)

In [ ]:
# df3

In [ ]:
# dfTickets_copy = dfTickets_copy.merge(df3, on = ['week_number','complainant_zone'], how = 'left')

In [ ]:
dfTickets_copy.info()

In [ ]:
dfTxns = prodFetch('''
select date_format(date, '%Y-%m') month, date, date_add(createdAt, interval 330 minute) txn_created_at, driverId complainant_id, partnerId partner_id, batteriesIssued batteries_issued from transactions
where date >= 20241001 and deletedAt is null
''')

In [ ]:
dfTxns

In [ ]:
replace_dict = {'[]':'', '[':'',']':'','"':''}
for old_char, new_char in replace_dict.items():
    dfTxns['batteries_issued'] = dfTxns['batteries_issued'].str.replace(old_char, new_char)
dfTxns[['B1_issued', 'B2_issued']] = dfTxns['batteries_issued'].str.split(',', expand = True)
dfTxns['complainant_id'] = dfTxns['complainant_id'].astype(str)
dfTickets_copy['complainant_id'] = dfTickets_copy['complainant_id'].astype(str)

In [ ]:
dfDriverZone = prodFetch("""select d.id complainant_id,
d.zoneId complainant_zone, 
(case when clientId not in ('BS00') then true else false end) b2b_2w_flag,
(case when clientId in ('BS00') and d.vehicleType not in ('E-2w') then true else false end) b2c_3w_flag,
(case when clientId in ('BS00') and d.vehicleType in ('E-2w') then true else false end) b2c_2w_flag
from drivers d
where d.id like 'D%'"""
)

In [ ]:
dfDriverZone

In [ ]:
dfTxns = dfTxns.merge(dfDriverZone, on = 'complainant_id', how = 'left')

In [ ]:
dfTxns

In [ ]:
df_zm_b2c_3w = dfTxns[dfTxns['b2c_3w_flag'] == 1]
df_zm_b2c_2w = dfTxns[dfTxns['b2c_2w_flag'] == 1]
df_zm_b2b_2w = dfTxns[dfTxns['b2b_2w_flag'] == 1]

In [ ]:
df_zm_b2c_3w = df_zm_b2c_3w.groupby(['month', 'complainant_zone']).agg( zone_monthly_B2C_3W_ops = ('complainant_id', 'nunique'), zone_monthly_B2C_3W_txns = ('complainant_id', 'count')).reset_index()
df_zm_b2c_2w = df_zm_b2c_2w.groupby(['month', 'complainant_zone']).agg( zone_monthly_B2C_2W_ops = ('complainant_id', 'nunique'), zone_monthly_B2C_2W_txns = ('complainant_id', 'count')).reset_index()
df_zm_b2b_2w = df_zm_b2b_2w.groupby(['month', 'complainant_zone']).agg( zone_monthly_B2B_2W_ops = ('complainant_id', 'nunique'), zone_monthly_B2B_2W_txns = ('complainant_id', 'count')).reset_index()

In [ ]:
df_zm_b2c_3w

In [ ]:
dfTickets_copy = dfTickets_copy.merge(df_zm_b2c_3w, on = ['month','complainant_zone'], how = 'left')
dfTickets_copy = dfTickets_copy.merge(df_zm_b2c_2w, on = ['month','complainant_zone'], how = 'left')
dfTickets_copy = dfTickets_copy.merge(df_zm_b2b_2w, on = ['month','complainant_zone'], how = 'left')

In [ ]:
dfTickets_copy

In [ ]:
df_zm_b2c_3w = dfTxns[dfTxns['b2c_3w_flag'] == 1]
df_zm_b2c_2w = dfTxns[dfTxns['b2c_2w_flag'] == 1]
df_zm_b2b_2w = dfTxns[dfTxns['b2b_2w_flag'] == 1]

In [ ]:
df_m_b2c_3w = df_zm_b2c_3w.groupby(['month']).agg( monthly_B2C_3W_ops = ('complainant_id', 'nunique'), monthly_B2C_3W_txns = ('complainant_id', 'count')).reset_index()
df_m_b2c_2w = df_zm_b2c_2w.groupby(['month']).agg( monthly_B2C_2W_ops = ('complainant_id', 'nunique'), monthly_B2C_2W_txns = ('complainant_id', 'count')).reset_index()
df_m_b2b_2w = df_zm_b2b_2w.groupby(['month']).agg( monthly_B2B_2W_ops = ('complainant_id', 'nunique'), monthly_B2B_2W_txns = ('complainant_id', 'count')).reset_index()

In [ ]:
df_zm_b2c_2w

In [ ]:
del df_zm_b2c_3w
del df_zm_b2c_2w
del df_zm_b2b_2w

In [ ]:
dfTickets_copy = dfTickets_copy.merge(df_m_b2c_3w, on = 'month', how = 'left')
dfTickets_copy = dfTickets_copy.merge(df_m_b2c_2w, on = 'month', how = 'left')
dfTickets_copy = dfTickets_copy.merge(df_m_b2b_2w, on = 'month', how = 'left')

In [ ]:
# df[df.ticketId == 'D240204-8283-1']

In [ ]:
# dfPrel = dfTickets_copy[dfTickets_copy['assigned_to_team']=="P-Rel"]

In [ ]:
# dfPrel[dfPrel['ticket_id']=='D240808-129143']

In [ ]:
# write("https://docs.google.com/spreadsheets/d/1pFystBfFB2YzuBwfOXAv_UpXX8dyt76XD66SPemIuSc/edit?gid=1988316138#gid=1988316138", "Raw_Data", dfPrel)

In [ ]:
# # daily ops drivers

# query7 = '''
# select t.date, 
#     count(distinct(case when d.clientId not in ('BS00') then driverId end)) day_B2B_2W_ops, 
#     count(distinct(case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) day_B2C_3W_ops, 
#     count(distinct(case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) day_B2C_2W_ops,
#     count((case when d.clientId not in ('BS00') then driverId end)) day_B2B_2W_txn, 
#     count((case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) day_B2C_3W_txns, 
#     count((case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) day_B2C_2W_txns
# from transactions t
# left join drivers d on d.id = t.driverId
# where t.date >= 20240701 and t.date <= current_date() and t.deletedAt is null
# group by 1
# '''

In [ ]:
# df7 = prodFetch(query7)

In [ ]:
# df7['date'] = pd.to_datetime(df7['date'])
# dfTickets_copy = dfTickets_copy.merge(df7, on = 'date', how = 'left')

In [ ]:
dfTickets_copy

In [ ]:
# # weekly Ops Drivers

# query8 = '''
# select week(t.date,1) week_number, 
#     count(distinct(case when d.clientId not in ('BS00') then driverId end)) week_B2B_2W_ops,
#     count(distinct(case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) week_B2C_3W_ops, 
#     count(distinct(case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) week_B2C_2W_ops,
#     count((case when d.clientId not in ('BS00') then driverId end)) week_B2B_2W_txns, 
#     count((case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) week_B2C_3W_txns, 
#     count((case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) week_B2C_2W_txns
# from transactions t
# left join drivers d on d.id = t.driverId
# where t.date >= 20240701 and t.date <= current_date() and t.deletedAt is null
# group by 1
# '''

In [ ]:
# df8 = prodFetch(query8)

In [ ]:
# dfTickets_copy = dfTickets_copy.merge(df8, on = 'week_number', how = 'left')

In [ ]:
dfTickets_copy

In [ ]:
dfTickets_copy

In [ ]:
# df11['date'] = pd.to_datetime(df11['date'])
# df11['hour'] = df11['hour'].apply(np.float64)
# dfTickets_copy = dfTickets_copy.merge(df11, on = ['date','hour'],  how = 'left')

In [ ]:
dfTickets_copy

In [ ]:
# # Zonal Weekly (Hourwise) Ops Drivers

# query12 = '''
# select week(t.date,1) week_number, hour(date_add(t.createdAt, interval 330 minute)) hour,
# 	d.zoneId complainant_zone,
# 	count(distinct(case when d.clientId not in ('BS00') then driverId end)) zone_hr_week_B2B_2W_ops, 
# 	count(distinct(case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) zone_hr_week_B2C_3W_ops, 
# 	count(distinct(case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) zone_hr_week_B2C_2W_ops,
#     count((case when d.clientId not in ('BS00') then driverId end)) zone_hr_week_B2B_2W_txns, 
# 	count((case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) zone_hr_week_B2C_3W_txns, 
# 	count((case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) zone_hr_week_B2C_2W_txns
# from transactions t
# left join drivers d on d.id = t.driverId
# where t.date >= 20240701 and t.date < current_date() and t.deletedAt is null
# group by 1,2,3
# '''

# # Weekly (Hourwise) Ops Drivers

# query13 = '''
# select week(t.date,1) week_number,
# 	hour(date_add(t.createdAt, interval 330 minute)) hour,
# 	count(distinct(case when d.clientId not in ('BS00') then driverId end)) hr_week_B2B_2W_ops, 
# 	count(distinct(case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) hr_week_B2C_3W_ops, 
# 	count(distinct(case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) hr_week_B2C_2W_ops,
#     count((case when d.clientId not in ('BS00') then driverId end)) hr_week_B2B_2W_txns, 
# 	count((case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) hr_week_B2C_3W_txns, 
# 	count((case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) hr_week_B2C_2W_txns 
# from transactions t
# left join drivers d on d.id = t.driverId
# where t.date >= 20240701 and t.date <= current_date() and t.deletedAt is null
# group by 1,2
# '''

In [ ]:
# df12 = prodFetch(query12)
# df13 = prodFetch(query13)

In [ ]:
# df12['hour'] = df12['hour'].apply(np.float64)
# dfTickets_copy = dfTickets_copy.merge(df12, on = ['week_number','hour', 'complainant_zone'], how = 'left')

In [ ]:
# dfTickets_copy

In [ ]:
# df13['hour'] = df13['hour'].apply(np.float64)
# dfTickets_copy = dfTickets_copy.merge(df13, on = ['week_number','hour'] ,how = 'left')

In [ ]:
dfTickets_copy

In [ ]:
# # Zonal Monthly (Hourwise) Ops Drivers 

# query14 = '''
# select date_format(t.date, '%Y-%m') month, hour(date_add(t.createdAt, interval 330 minute)) hour,
# 	d.zoneId complainant_zone,
# 	count(distinct(case when d.clientId not in ('BS00') then driverId end)) zone_hr_monthly_B2B_2W_ops, 
# 	count(distinct(case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) zone_hr_monthly_B2C_3W_ops, 
# 	count(distinct(case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) zone_hr_monthly_B2C_2W_ops,
#     count((case when d.clientId not in ('BS00') then driverId end)) zone_hr_monthly_B2B_2W_txns, 
# 	count((case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) zone_hr_monthly_B2C_3W_txns, 
# 	count((case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) zone_hr_monthly_B2C_2W_txns 
# from transactions t
# left join drivers d on d.id = t.driverId
# where t.date >= 20240701 and t.date <= current_date() and t.deletedAt is null
# group by 1,2,3
# '''

# # Monthly (Hourwise) Ops Drivers

# query15 = '''
# select date_format(t.date, '%Y-%m') month, hour(date_add(t.createdAt, interval 330 minute)) hour,
# 	count(distinct(case when d.clientId not in ('BS00') then driverId end)) hr_monthly_B2B_2W_ops, 
# 	count(distinct(case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) hr_monthly_B2C_3W_ops, 
# 	count(distinct(case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) hr_monthly_B2C_2W_ops,
#     count((case when d.clientId not in ('BS00') then driverId end)) hr_monthly_B2B_2W_txns, 
# 	count((case when d.clientId in ('BS00') and d.vehicleType not in ('E-2w') then driverId end)) hr_monthly_B2C_3W_txns, 
# 	count((case when d.clientId in ('BS00') and d.vehicleType in ('E-2w') then driverId end)) hr_monthly_B2C_2W_txns 
# from transactions t
# left join drivers d on d.id = t.driverId
# where t.date >= 20240701 and t.date <= current_date() and t.deletedAt is null
# group by 1,2
# '''

In [ ]:
# df14 = prodFetch(query14)
# df15 = prodFetch(query15)

In [ ]:
# df14['hour'] = df14['hour'].apply(np.float64)
# dfTickets_copy = dfTickets_copy.merge(df14, on = ['month','hour', 'complainant_zone'], how = 'left')

In [ ]:
dfTickets_copy

In [ ]:
# df15['hour'] = df15['hour'].apply(np.float64)
# dfTickets_copy = dfTickets_copy.merge(df15, on = ['month','hour'] , how = 'left')

In [ ]:
dfTickets_copy

In [ ]:
dfTxnsGr = prodFetch("""select driverId complainant_id, count(DISTINCT(date)) txn_days from transactions
where (date >= date_add(current_date(), interval -90 day) and date < current_date()) and deletedAt is null -- and clientId in ('BS00')
group by 1""")

In [ ]:
dfActiveDays = prodFetch("""select driverId complainant_id, count(case when status in ('active') then driverId end) active_days, count(date) total_days from driverStatusHistories
where (date >= date_add(current_date(), interval -90 day) and date < current_date()) and deletedAt is null and driverId like 'D%'
group by 1""")

In [ ]:
dfActiveDays = dfActiveDays.merge(dfTxnsGr, on = 'complainant_id', how = 'left')

In [ ]:
dfActiveDays = dfActiveDays.fillna(0)

In [ ]:
dfActiveDays['active_days'] = np.where(dfActiveDays['active_days'] < dfActiveDays['txn_days'], dfActiveDays['txn_days'],  dfActiveDays['active_days'])

In [ ]:
dfActiveDays['txn_days/active_days'] =  (dfActiveDays['txn_days']/dfActiveDays['active_days']).round(2)

In [ ]:
dfActiveDays

In [ ]:
# To define txn/active days cohort

conditions = [
    ((dfActiveDays['txn_days/active_days'] >= 0) & (dfActiveDays['txn_days/active_days'] <=0.10)),
    ((dfActiveDays['txn_days/active_days'] > 0.10) & (dfActiveDays['txn_days/active_days'] <=0.25)),
    ((dfActiveDays['txn_days/active_days'] > 0.25) & (dfActiveDays['txn_days/active_days'] <=0.5)),
    ((dfActiveDays['txn_days/active_days'] > 0.5) & (dfActiveDays['txn_days/active_days'] <=0.75)),
    ((dfActiveDays['txn_days/active_days'] > 0.75) & (dfActiveDays['txn_days/active_days'] <=0.9)),
    ((dfActiveDays['txn_days/active_days'] > 0.9))
]

# Define the corresponding output for each condition
choices = ['A(0% - 10%)', 'B(11% - 25%)', 'C(26% - 50%)', 'D(51% - 75%)', 'E(76% - 90%)', 'F(91% - 100%)' ]

# Apply the conditions to create the 'age_cohort' column
dfActiveDays['txn_active_cohort'] = np.select(conditions, choices, default='')

In [ ]:
dfActiveDays

In [ ]:
dfTickets_copy = dfTickets_copy.merge(dfActiveDays, on = 'complainant_id', how = 'left')

In [ ]:
reqCols = ['date', 'txn_created_at', 'complainant_id', 'partner_id','B1_issued', 'B2_issued' ]
dfTxns = dfTxns[reqCols]

In [ ]:
dfTickets_copy

In [ ]:
dfTxns['complainant_id'] = dfTxns['complainant_id'].astype(str)
dfTickets_copy['complainant_id'] = dfTickets_copy['complainant_id'].astype(str)

In [ ]:
dfTxns.head()

In [ ]:
dfTickets_copy

In [ ]:
dfTxns = dfTxns.sort_values(by = ['txn_created_at', 'complainant_id'])

In [ ]:
dfTxns

In [ ]:
dfTickets_copy = dfTickets_copy.sort_values(by = ['created_at', 'complainant_id'])

In [ ]:
dfTickets_copy = pd.merge_asof(dfTickets_copy, dfTxns, left_on = 'created_at', right_on = 'txn_created_at', by = 'complainant_id', direction = 'backward', suffixes=('', '_prev'))

In [ ]:
dfTxns

In [ ]:
dfTickets_copy

In [ ]:
dfTickets_copy = dfTickets_copy.sort_values(by = ['created_at', 'complainant_id'])

In [ ]:
# # Ensure the indexes are reset after sorting
# dfTxns.reset_index(drop=True, inplace=True)
# dfTickets_copy.reset_index(drop=True, inplace=True)

In [ ]:
dfTickets_copy

In [ ]:
dfTxns.head()

In [ ]:
dfTickets_copy = pd.merge_asof(dfTickets_copy,dfTxns, left_on = 'created_at', right_on = 'txn_created_at', by = 'complainant_id', direction = 'forward', suffixes=('', '_next'))

In [ ]:
dfTickets_copy

In [ ]:
# dfMultipleTickets = dfTickets_copy.groupby(['date', 'complainant_id', 'category_issue']).agg({'ticket_id':'count'}).reset_index()

In [ ]:
# dfMultipleTickets.rename({'ticket_id':'same_day_ticket_count'}, axis = 1, inplace = True)

In [ ]:
# dfTickets_copy = dfTickets_copy.merge(dfMultipleTickets, on = ['date', 'complainant_id'], how = 'left')

In [ ]:
# dfTickets_copy

In [ ]:
# dfTickets_copy['L7D_tickets'] = 0
# dfTickets_copy['L14D_tickets'] = 0
# dfTickets_copy['L30D_tickets'] = 0
# dfTickets_copy['L60D_tickets'] = 0
# dfTickets_copy['L90D_tickets'] = 0
# dfTickets_copy['C0_choking'] = 0
# dfTickets_copy['C1_choking'] = 0

In [ ]:
# dfTickets_copy = dfTickets_copy.fillna(0)

In [ ]:
# dfChokingScore = adbFetch('')

In [ ]:
# dfTickets_copy['remark'] = dfTickets_copy['remark'].astype(str) 
# dfTickets_copy['rejectionremarks'] = dfTickets_copy['rejectionremarks'].astype(str)

In [ ]:
dfTickets_copy['remark'] = dfTickets_copy['remark'].str.replace('+', ' ')
# dfTickets_copy['remark'] = dfTickets_copy['remark'].str.replace(' ', '')
dfTickets_copy['remark'] = dfTickets_copy['remark'].apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8') if isinstance(x, str) else x)

In [ ]:
dfTickets_copy['remark'] = dfTickets_copy['remark'].str.replace('+', ' ')
# dfTickets_copy['remark'] = dfTickets_copy['remark'].str.replace(' ', '')
dfTickets_copy['remark'] = dfTickets_copy['remark'].apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8') if isinstance(x, str) else x)

In [ ]:
dfTickets_copy['rejectionremarks'] = dfTickets_copy['rejectionremarks'].str.replace('+', ' ')
# dfTickets_copy['rejectionremarks'] = dfTickets_copy['rejectionremarks'].str.replace(' ', '')
dfTickets_copy['rejectionremarks'] = dfTickets_copy['rejectionremarks'].apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8') if isinstance(x, str) else x)

In [ ]:
# dfTickets_copy.iloc[1:20].to_clipboard()

In [ ]:
dfTickets_copy

In [ ]:
dfTickets_copy['date'] = pd.to_datetime(dfTickets_copy['date']).dt.date

In [ ]:
dfTickets_copy['date_next'] = pd.to_datetime(dfTickets_copy['date_next'])

In [ ]:
# dfTickets_copy['date_next']

In [ ]:
# del dfTickets_copy['category_issue_y']
# del dfTickets_copy['batteries_issued']
# del dfTickets_copy['batteries_issued_next']

In [ ]:
dfTickets_copy.head()

In [ ]:
renameCols = {'category_issue_x':'category_issue', 'txn_created_at':'txn_created_at_prev','B1_issued':'B1_issued_prev', 'B2_issued':'B2_issued_prev'}

dfTickets_copy.rename(renameCols, axis = 1, inplace = True)

In [ ]:
dfTickets_copy['now'] = pd.Timestamp.now()

In [ ]:
import sys, os, pandas as pd, numpy as np, json, pygsheets as pg
sys.path.append('../connectors')
sys.path.append('../utils')
from datetime import datetime, date, timedelta, timezone

from matplotlib import pyplot as plt

from tqdm import tqdm
from queryHelper import *

import warnings 
warnings.filterwarnings('ignore')

import h3

locationsDf = prodMongoFetch(collection= 'driverAggregateLocations', pipeline= []).drop(columns = ['_id'])
locationsDf['mpl'] = locationsDf['locations'].apply(lambda x : x[0])

locationsDf['lat'] = locationsDf['mpl'].apply(lambda x : x.get('lat'))
locationsDf['lon'] = locationsDf['mpl'].apply(lambda x : x.get('lon'))
locationsDf['instances'] = locationsDf['mpl'].apply(lambda x : x.get('instances'))
locationsDf['probability'] = locationsDf['mpl'].apply(lambda x : x.get('probability'))

del(locationsDf['locations'])
del(locationsDf['mpl'])

locationsDf = locationsDf[(locationsDf.probability > 0.5) | (locationsDf.instances > 15)].sort_values(by = 'driverId').reset_index(drop = True)

locationsDf['driverH3Id'] = locationsDf.apply(lambda row : h3.geo_to_h3(row['lat'], row['lon'], 7), axis = 1)

locationsDf.rename({'driverId':'complainant_id'}, axis = 1, inplace = True)

# del locationsDf['lat']
# del locationsDf['lon']
# del locationsDf['instances']
# del locationsDf['probability']

locationsDf

In [ ]:
dfTickets_copy = dfTickets_copy.merge(locationsDf, on = 'complainant_id', how = 'left')

In [ ]:
dfTickets_copy = dfTickets_copy.fillna("")

In [ ]:
dfTickets_copy

In [ ]:
del locationsDf

In [ ]:
dfTickets_copy.rename({'assinged_to_team':'assigned_to_role_team'}, axis = 1, inplace = True)

In [ ]:
dfTickets_copy

In [ ]:
# conn = psycopg2.connect(dbname = "operations_manager_prod", user = "ankit_das", password = "Ankit@12345", host = "operation.replica.upgrid.in", port = "5432")

# print('PostgreSQL Connection Established')

# # Fetch Data from PostgreSQL server

# query1 = '''with v1 as 
# (
# select
#     TO_CHAR((t.created_at + INTERVAL '330 minute')::date, 'YYYY-MM') as month,
#     EXTRACT(WEEK FROM (t.created_at + INTERVAL '330 minute')::date) AS week_number,
#     (case 
#     	when (EXTRACT(DAY FROM (t.created_at + INTERVAL '330 minute')::date) >= 1 and EXTRACT(DAY FROM (t.created_at + INTERVAL '330 minute')::date) <= 10) then '1-10' 
#     	when (EXTRACT(DAY FROM (t.created_at + INTERVAL '330 minute')::date) >= 11 and EXTRACT(DAY FROM (t.created_at + INTERVAL '330 minute')::date) <= 20) then '11-20'
#     	when EXTRACT(DAY FROM (t.created_at + INTERVAL '330 minute')::date) >= 21 then '21+' 
#     end) day_cohort,
#     to_char((t.created_at + INTERVAL '330 minute')::date, 'Day') as day,
#     (t.created_at + INTERVAL '330 minute')::date AS date,
#     t.created_at + INTERVAL '330 minute' AS created_at,
#     t.updated_at + INTERVAL '330 minute' AS updated_at,
#     TO_CHAR(t.created_at + INTERVAL '330 minutes', 'HH24') as hour,
#     t.slug ticket_id,
#     tasks.status ticket_status,
#     (case when t.complainant_type = 1 then 'driver' else 'partner' end) complainant_type,
#     t.complainant_id complainant_id,
#     ic.id category_id,
#     ic.name category_name,
#     t.issue_id issue_id,
#     i1.name issue_name,
#     concat(ic.name,' - ',i1.name) category_issue,
#     t.complainant_name complainant_name,
#     t.zone complainant_zone,
#     t.source,
#     COALESCE(t.calling_number, '') calling_number,
#     round(EXTRACT(EPOCH FROM (t.sla - t.created_at))/3600,0) sla_in_hrs,
#     y.done_by created_by,
#     (case when iar.assignment_rule_type like '%TeamRole%' then 'Team Role' else 'RM' end) assingment_role,
#     tarm.team rmTeam,
#     COALESCE((case when iar.assignment_rule_type like '%TeamRole%' then tatr.team else NULL end),' ') assigned_to_team,
#     COALESCE((case when iar.assignment_rule_type like '%TeamRole%' then tatr.role else NULL end ), '') assigned_to_role,
#     tasks.id taskId,
#     COALESCE(tasks.location, '') taskLocation,
#     COALESCE(x.rejectionCount,0) taskRejectionCount,
#     COALESCE(x.rejectionReasons,'') rejectionReasons,
#     COALESCE(x.rejectionRemarks,'') rejectionRemarks,
#     battery_ids,
#     split_part(battery_ids, ',', 1) AS batteryid_1,
#     split_part(battery_ids, ',', 2) AS batteryid_2,
#     charger_ids,
#     partner_id, 
#     driver_id
# from tickets t
# left join issues i1 on t.issue_id = i1.id
# left join issue_categories ic on ic.id = i1.issue_category_id
# left join tasks on tasks.id = t.task_id
# left join issue_assignment_rules iar on iar.issue_id = t.issue_id
# left join tickets_assignment_rules_relationship_managers tarm on tarm.id = iar.assignment_rule_id
# left join tickets_assignment_rules_team_roles tatr on tatr.id = iar.assignment_rule_id
# left join (
# with v1 as 
# (
# select loggable_type, loggable_id, to_value, count(id) rejectionCount, string_agg(reason, E' >> ') rejectionReasons, string_agg(remark, E' >> ') rejectionRemarks from logs
# where to_value = 'rejected'
# group by 1,2,3
# )
# select * from v1) x on x.loggable_id = t.task_id
# left join (
# with logData as 
# (
# select loggable_type, loggable_id, from_value, to_value, reason, remark, done_by, event_name, ROW_NUMBER() over(partition by loggable_id order by created_at) rank from logs 
# )
# select * from logData 
# where rank = 1 and loggable_type in ('Task')
# ) y on y.loggable_id = t.task_id
# where t.deleted_at is null
# )
# select * from v1
# '''

# dfTickets2 = pd.read_sql(query1, conn)


# # dfTickets

# conn.close()

In [ ]:
# dfTickets2

In [ ]:
time.sleep(10)

In [ ]:
# conn = psycopg2.connect(dbname = "operations_manager_prod", user = "ankit_das", password = "Ankit@12345", host = "operation.replica.upgrid.in", port = "5432")

# print('PostgreSQL Connection Established')

# query3 = '''
# select l.loggable_type, l.loggable_id, l.attr, l.from_value, l.to_value, l.reason, l.remark, l.event_name, l.done_by,  l.created_at + INTERVAL '330 minute' created_at, l.updated_at + INTERVAL '330 minute' updated_at,
# COALESCE(t.parent_id,l.loggable_id) parent_Id
# from logs l
# left join tasks t on l.loggable_id = t.id 
# where deleted_at is null  and (t.created_at + INTERVAL '330 minute')::date >= '20240801' and l.remark not like 'Assigned to%' and l.remark != ''
# '''

# dfTat = pd.read_sql(query3, conn)

# conn.close()

In [ ]:
# dfTat['created_at'] = pd.to_datetime(dfTat['created_at'])
# dfTat['updated_at'] = pd.to_datetime(dfTat['updated_at'])
# dfTat.rename({'parent_id':'taskid'}, axis = 1, inplace = True)

# dfUsers = prodFetch("""select employeeId temp, (case when teams in ('[]') then role else teams end) team, roleV2 from users 
# where employeeId not like 'D%' or employeeId not like 'P%'""")

# replace_dict = {'[]':'', '[':'',']':'','"':''}

# dfUsers['roleV2'] = dfUsers['roleV2'].fillna('')

# dfUsers['assinged_to_team'] = dfUsers['team'] + " - " + dfUsers['roleV2']

# del dfUsers['team']
# del dfUsers['roleV2']

# dfTat['temp'] = dfTat['done_by'].str[:8].str.replace(' ', '')

# dfTat = dfTat.merge(dfUsers, on = 'temp', how = 'left')

# dfTat.rename({'temp':'done_by_id'}, axis = 1, inplace = True)

# dfOSC = ['operationsSupportTeam - centralAssociate','operationsSupportTeam - centralAssociate','["centralOsc"] - fieldExecutive','["centralOsc"] - centralAssociate','operationsSupportTeam - centralTeamLead','["centralOsc"] - admin',
# ]

# dfOscRemarks = dfTat[dfTat['assinged_to_team'].isin(dfOSC)]

# dfOscRemarks = dfOscRemarks.merge(dfTickets2, on = 'taskid', how = 'left')

# dfRemarks = dfOscRemarks.groupby(['ticket_id']).agg({'remark':'count'}).reset_index()

# dfRemarks.rename({'remark':'osc_remarks'}, axis = 1, inplace = True)

In [ ]:
# dfRemarks

In [ ]:
# dfTickets_copy

In [ ]:
# dfTickets_copy = dfTickets_copy.merge(dfRemarks, on = 'ticket_id', how = 'left')

In [ ]:
# dfTickets_copy['osc_remarks'] = dfTickets_copy['osc_remarks'].fillna(0)

In [ ]:
# dfTickets_copy

To Get If The Vehicle is Registered or Not

In [ ]:
# dfVehicleReg = prodFetch("""with v1 as 
# (
# select driverId, registrationNumber, registrationDate, date(fitnessCertificateExpiryDate) fitnessCertificateExpiryDate from driverVehicleVerificationLogs
# union
# select driverId, registrationNo, date(registrationDate) registrationDate, NULL fitnessCertificateExpiryDate  from driverOnboardingDatas
# where driverId is not null
# ),
# prefinal as 
# (
# select driverId, GROUP_concat(registrationNumber) registrationNumber, date(max(registrationDate)) registrationDate, date(max(fitnessCertificateExpiryDate)) fitnessCertificateExpiryDate from v1
# group by 1
# ),
# prefinal2 as 
# (
# select driverId,
# 	   registrationNumber,
# 	   (case when registrationDate < 20100101 then NULL else registrationDate end) registrationDate
# from prefinal
# )
# select a.*,  date_add(registrationDate, interval 2 year) fitnessCertificateExpiryDate, (case when registrationDate is not null then 1 else 0 end) registrationFlag, (case when date_add(registrationDate, interval 2 year) > current_date() then 1 else 0 end) vehicleFitnessFlag
# from prefinal2 a
# left join drivers d on a.driverId = d.id
# """)

In [ ]:
# dfVehicleReg

In [ ]:
# dfTickets_copy = dfTickets_copy.merge(dfVehicleReg, left_on = 'complainant_id', right_on = 'driverId', how = 'left')

In [ ]:
# dfTickets_copy['registrationFlag'] = dfTickets_copy['registrationFlag'].fillna("") 
# dfTickets_copy['vehicleFitnessAvailable'] = dfTickets_copy['vehicleFitnessAvailable'].fillna("") 

In [ ]:
# del dfTickets_copy['driverId']

In [ ]:
# dfTickets_copy

Driver Age Cohort Basis Ticket Date

In [ ]:
del dfTickets_copy['age_cohort']

dfTickets_copy['date'] = pd.to_datetime(dfTickets_copy['date'])
dfTickets_copy['driver_live_date'] = pd.to_datetime(dfTickets_copy['driver_live_date'])

In [ ]:
dfTickets_copy['date_diff'] = (dfTickets_copy['date'] - dfTickets_copy['driver_live_date']).dt.days


In [ ]:
dfTickets_copy['date_diff'] = dfTickets_copy['date_diff'].fillna(0)

# dfTickets_copy['date_diff'] = np.where(dfTickets_copy['date_diff'] == "", 0, dfTickets_copy['date_diff'])

In [ ]:
dfTickets_copy

In [ ]:
# Define conditions for the age cohort
conditions = [
    (dfTickets_copy['date_diff'] <= 30),
    (dfTickets_copy['date_diff'] >= 31) & (dfTickets_copy['date_diff'] <= 90),
    (dfTickets_copy['date_diff'] > 90) & (dfTickets_copy['date_diff'] <= 365),
    (dfTickets_copy['date_diff'] > 365)
]

# Define the corresponding output for each condition
choices = ['<= 30 days', '31-90 days', '91-365 days', '365+ days']

# Apply the conditions to create the 'age_cohort' column
dfTickets_copy['age_cohort'] = np.select(conditions, choices, default='')

# Drop the date_diff column if you no longer need it
dfTickets_copy.drop(columns=['date_diff'], inplace=True)

In [ ]:
dfTickets_copy

In [ ]:
dfTickets_copy['age_cohort'] = np.where(dfTickets_copy['complainant_type']=='partner', "", dfTickets_copy['age_cohort']  )

In [ ]:
dfTickets_copy

In [ ]:
# dfTicketsVehicle = dfTickets[dfTickets['category_name']=='Vehicle Issue']

In [ ]:
# dfTicketsVehicle

In [ ]:
dfTickets_copy

In [ ]:
# (dfTickets_copy.groupby('ticket_id').agg({'category_issue':'count'}).reset_index()).to_clipboard()

In [ ]:
# dfTicketsVehicle.groupby(['month','complainant_zone']).agg({'ticket_id':'count', 'osc_remarks':'sum'})

In [ ]:
dfCallingNumber = prodFetch("""with v1 as 
(
select customerId, callingNumber, count, row_number() over(partition by customerId order by count desc) rank  from customerCallingNumbers
where deletedAt is null
-- group by 1,2
)
select customerId complainant_id, callingNumber primary_number from v1
where rank = 1""")

dfTickets_copy = dfTickets_copy.merge(dfCallingNumber, on ='complainant_id', how = 'left')

In [ ]:
# del dfCallingNumber

In [ ]:
dfTickets_copy.info()

In [ ]:
dfTickets_copy['date'] = dfTickets_copy['date'].dt.date
dfTickets_copy['driver_live_date'] = pd.to_datetime(dfTickets_copy['driver_live_date']).dt.date
dfTickets_copy['date_next'] = pd.to_datetime(dfTickets_copy['date_next']).dt.date

In [ ]:
dfTickets_copy

In [ ]:
from sqlalchemy import create_engine
from urllib.parse import quote

user = os.getenv('SQL_USER2')
password = quote(os.getenv('SQL_PWD2'), safe ='')
url = os.getenv('SQL_HOST2')
databaseUrl = f"mysql+pymysql://{user}:{password}@{url}/analytics_prod?charset=utf8mb4"

engine = create_engine(databaseUrl)
dfTickets_copy.to_sql('ticketModelNew', con = engine, if_exists = 'replace', index = False, dtype = {'remark': sqlalchemy.types.TEXT(collation='utf8mb4_unicode_ci'), 
'rejectionremarks': sqlalchemy.types.TEXT(collation='utf8mb4_unicode_ci'), 'tasklocation': sqlalchemy.types.TEXT(collation='utf8mb4_unicode_ci')})

To Write B2B TMS data  

In [ ]:
from sheetHelper import *

In [ ]:
dfB2B = adbFetch('''

select distinct(ticket_id), date, month, week_number, day, created_at, updated_at, ticket_status, 
complainant_type, complainant_id, category_name, issue_name, category_name, complainant_name, source, calling_number, taskrejectioncount,partner_id, complainant_zone, resolution, remark, 
client_id, client,left(complainant_zone,3) as region,
case when LEFT(complainant_zone, 3) in ('NWD','NMV','SHG','FSD','DNG') then 'NCR' else LEFT(complainant_zone, 3) end as region2

from ticketModelNew
where b2b_2w_flag = 1
order by date desc
'''
)

write('https://docs.google.com/spreadsheets/d/1W_pl-LND1ED1xqpfLPGDrphGak-8bRWAfCbhwgS3fMI/edit?gid=505628546#gid=505628546', "TMS Raw", dfB2B)